# Underfitting, overfitting, regularization, dropout

## Load and minibatch MNIST data

In [ ]:
using Knet, Plots

In [ ]:
include(Knet.dir("examples","mnist.jl"))
MNIST.loaddata()
using MNIST: xtrn,ytrn,xtst,ytst,minibatch
Atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32}
dtst = minibatch(xtst,ytst,100;atype=Atype) # [ (x1,y1), (x2,y2), ... ] where xi,yi are minibatches of 100
dtrn = minibatch(xtrn,ytrn,100;atype=Atype) # [ (x1,y1), (x2,y2), ... ] where xi,yi are minibatches of 100
length(dtrn),length(dtst)

In [ ]:
(x,y)=dtst[1]

In [ ]:
display(x)

In [ ]:
display(y)

## Define linear model

In [ ]:
function linear(w,x)
    y = w[1]*x .+ w[2]
end

In [ ]:
winit()=map(Atype, [ 0.01*randn(10,784), zeros(10,1) ])
w = winit()  # random weight matrix and a zero bias vector

In [ ]:
ypred=linear(w,x)
display(ypred)

In [ ]:
display(y)

## Measuring error rate

In [ ]:
sum(y .* (ypred .== maximum(ypred,1)))  # number of correct answers in this batch of 100 with initial w

In [ ]:
function zeroone(w,data,model)
    ncorr = ninst = 0
    for (x,y) in data
        ypred = model(w,x)
        ncorr += sum(y .* (ypred .== maximum(ypred,1)))
        ninst += size(x,2)
    end
    return 1 - ncorr/ninst
end

In [ ]:
zeroone(w,dtst,linear)  # number of errors in the whole test dataset

## Measuring loss

In [ ]:
# Calculate cross entropy loss of a model with weights w for one minibatch (x,p)
# Use non-zero l1 or l2 for regularization
function softloss(w,x,p,model;l1=0,l2=0,o...)  
    y = model(w,x;o...)
    y = y .- maximum(y,1) # for numerical stability
    expy = exp(y)
    logphat = y .- log(sum(expy,1))
    J = -sum(p .* logphat) / size(x,2)  # dividing by number of instances for per-instance average
    if l1 != 0; J += l1 * sum(sumabs(wi)  for wi in w[1:2:end]); end
    if l2 != 0; J += l2 * sum(sumabs2(wi) for wi in w[1:2:end]); end
    return J
end

In [ ]:
softloss(w,x,y,linear)  # per-instance average softloss for the first test minibatch

In [ ]:
function avgloss(w,data,model) # average loss for the whole dataset
    sum = cnt = 0
    for (x,y) in data
        sum += softloss(w,x,y,model)
        cnt += 1
    end
    return sum/cnt
end

In [ ]:
avgloss(w,dtst,linear)  # per-instance average softloss for the whole test set

## Manually defined gradient for softloss

In [ ]:
function grad1(w,x,p,model)
    y = model(w,x)
    y = y .- maximum(y,1) # for numerical stability
    expy = exp(y)
    q = expy ./ sum(expy,1)
    dJdy = (q - p) / size(x,2)
    dJdw = dJdy * x'
    dJdb = sum(dJdy,2)
    Any[dJdw,dJdb]
end

## Automatically defined gradient for softloss

In [ ]:
softgrad = grad(softloss)  # Knet/AutoGrad makes life easier :)

In [ ]:
g1 = grad1(w,x,y,linear)

In [ ]:
g2 = softgrad(w,x,y,linear)

In [ ]:
isapprox(g1[1],g2[1])

In [ ]:
isapprox(g1[2],g2[2])

## Checking the gradient

In [ ]:
display(g1[2])  
# Meaning of gradient:
# If I move the [8] entry by epsilon, loss will go up by 0.306271 epsilon!

In [ ]:
display(w[2])

In [ ]:
softloss(w,x,y,linear)

In [ ]:
w[2][8] = 0.1   # to numerically check the gradient let's move entry 8 by +0.1.

In [ ]:
softloss(w,x,y,linear)  
# we see that the loss moves by +0.03 as expected.  Normally we would check all entries or a large sample.

## Training loop (SGD)

In [ ]:
# Train model(w) with SGD and return a list containing w at the end of each epoch
function train(w,data,model; epochs=100,lr=0.5,o...)
    weights = Any[copy(w)]
    for epoch in 1:epochs
        for (x,y) in data
            g = softgrad(w,x,y,model;o...)
            for i in 1:length(w)
                w[i] = w[i] - lr * g[i]
            end
        end
        push!(weights,copy(w))
    end
    return weights
end

## Training the linear model

In [ ]:
srand(1)
@time trn1=train(winit(),dtrn,linear,lr=0.1)
@time trnloss1 = [ avgloss(w,dtrn,linear) for w in trn1 ]
@time tstloss1 = [ avgloss(w,dtst,linear) for w in trn1 ]
@time trnerr1 = [ zeroone(w,dtrn,linear) for w in trn1 ]
@time tsterr1 = [ zeroone(w,dtst,linear) for w in trn1 ]
minimum(tstloss1),minimum(tsterr1)

In [ ]:
length(trn1)

## Overfitting and underfitting

In [ ]:
plot([trnloss1 tstloss1],ylim=(.2,.36),labels=[:trnloss :tstloss],xlabel="Epochs",ylabel="Loss") 
# Demonstrates both overfitting and underfitting
# Overfitting: test loss is higher than training loss and getting worse
# Underfitting: training loss not close to 0

In [ ]:
plot([trnerr1 tsterr1],ylim=(.06,.10),labels=[:trnerr :tsterr],xlabel="Epochs",ylabel="Error")  
# this is the error plot, we get to about 8% error, i.e. 92% accuracy

## Linear model with L2 regularization

In [ ]:
# Regularization may help overfitting
srand(1)
@time trn2=train(winit(),dtrn,linear;l2=0.00004)
@time trnloss2 = [avgloss(w,dtrn,linear) for w in trn2]
@time tstloss2 = [avgloss(w,dtst,linear) for w in trn2]
@time trnerr2 = [zeroone(w,dtrn,linear) for w in trn2]
@time tsterr2 = [zeroone(w,dtst,linear) for w in trn2]
minimum(tstloss2)

In [ ]:
plot([trnloss1 tstloss1 trnloss2 tstloss2],ylim=(0.2,0.36),
    labels=[:trnloss :tstloss :trnlossL2 :tstlossL2],xlabel="Epochs",ylabel="Loss") 
# overfitting less but results do not improve much

In [ ]:
plot([trnerr1 tsterr1 trnerr2 tsterr2],ylim=(0.06,0.10),
    labels=[:trnerr :tsterr :trnerrL2 :tsterrL2],xlabel="Epochs",ylabel="Error")

## Multi-layer Perceptron

In [ ]:
# Using a model with higher capacity helps underfitting
function mlp(w,x)
    for i=1:2:length(w)-2
        x = relu(w[i]*x .+ w[i+1])
    end
    return w[end-1]*x .+ w[end]
end

In [ ]:
function winit(h...; std=0.01, x=784, y=10,  # use winit(h1,h2,...,hn) for n hidden layer mlp
               atype=gpu()>=0 ? KnetArray{Float32} : Array{Float32})
    h = [x, h..., y]
    w = Any[]
    for i=1:length(h)-1
        push!(w, std*randn(h[i+1],h[i]))
        push!(w, zeros(h[i+1],1))
    end
    map(atype, w)
end

In [ ]:
w2=winit(64) # gives weights and biases for an MLP with a single hidden layer of size 64

In [ ]:
softloss(w2,x,y,mlp)

In [ ]:
srand(1)
@time trn3=train(winit(64),dtrn,mlp)
@time trnloss3 = [ avgloss(w,dtrn,mlp) for w in trn3 ]
@time tstloss3 = [ avgloss(w,dtst,mlp) for w in trn3 ]
@time trnerr3 = [ zeroone(w,dtrn,mlp) for w in trn3 ]
@time tsterr3 = [ zeroone(w,dtst,mlp) for w in trn3 ]
minimum(tstloss3)

In [ ]:
plot([trnloss1 tstloss1 trnloss3 tstloss3],ylim=(0.0,0.4),
    labels=[:trnLin :tstLin :trnMLP :tstMLP],xlabel="Epochs",ylabel="Loss")  
# solves the underfitting problem!

In [ ]:
plot([trnerr1 tsterr1 trnerr3 tsterr3],ylim=(0,0.1),
    labels=[:trnLin :tstLin :trnMLP :tstMLP],xlabel="Epochs",ylabel="Error")  
# error improves from 8% to 2%!

## MLP with L1 regularization

In [ ]:
# We still have overfitting, let's try regularization again
srand(1)
@time trn4=train(winit(64),dtrn,mlp;lr=0.1,epochs=100,l1=4e-5)
@time trnloss4 = [ avgloss(w,dtrn,mlp) for w in trn4 ]
@time tstloss4 = [ avgloss(w,dtst,mlp) for w in trn4 ]
@time trnerr4 = [ zeroone(w,dtrn,mlp) for w in trn4 ]
@time tsterr4 = [ zeroone(w,dtst,mlp) for w in trn4 ]
minimum(tstloss4)

In [ ]:
plot([trnloss3 tstloss3 trnloss4 tstloss4],ylim=(0,0.15),
    labels=[:trnMLP :tstMLP :trnMLP_L1 :tstMLP_L1],xlabel="Epochs", ylabel="Loss")  
# overfitting less, loss results improve

In [ ]:
plot([trnerr3 tsterr3 trnerr4 tsterr4],ylim=(0,0.04),
    labels=[:trnMLP :tstMLP :trnMLP_L1 :tstMLP_L1],xlabel="Epochs", ylabel="Error")    
# however error results do not improve! 

In [ ]:
minimum(tsterr3),minimum(tsterr4)

## MLP with dropout

In [ ]:
# Dropout is another way to address overfitting
function dropout(x,p)
    if p > 0
        x .* (rand!(similar(x)) .> p)
    else
        x
    end
end

function mlp(w,x; pdrop=(0,0))
    x = dropout(x,pdrop[1])
    for i=1:2:length(w)-2
        x = relu(w[i]*x .+ w[i+1])
        x = dropout(x,pdrop[2])
    end
    return w[end-1]*x .+ w[end]
end

In [ ]:
srand(1)
@time trn5=train(winit(64),dtrn,mlp;lr=0.1,epochs=100,pdrop=(0.2,0))
@time trnloss5 = [ avgloss(w,dtrn,mlp) for w in trn5 ]
@time tstloss5 = [ avgloss(w,dtst,mlp) for w in trn5 ]
@time trnerr5 = [ zeroone(w,dtrn,mlp) for w in trn5 ]
@time tsterr5 = [ zeroone(w,dtst,mlp) for w in trn5 ]
minimum(tstloss5),minimum(tsterr5)

In [ ]:
plot([trnloss3 tstloss3 trnloss5 tstloss5],ylim=(0,0.15),
    labels=[:trnMLP :tstMLP :trnDropout :tstDropout],xlabel="Epochs", ylabel="Loss")  
# overfitting less, loss results improve

In [ ]:
plot([trnerr3 tsterr3 trnerr5 tsterr5],ylim=(0,0.04),
    labels=[:trnMLP :tstMLP :trnDropout :tstDropout],xlabel="Epochs", ylabel="Error")  
# this time error also improves slightly

In [ ]:
minimum(tsterr3),minimum(tsterr4),minimum(tsterr5)

In [ ]:
minimum(tstloss3),minimum(tstloss4),minimum(tstloss5)

## MLP with larger hidden layer

In [ ]:
# The current trend is to use models with higher capacity tempered with dropout
srand(1)
@time trn6=train(winit(256),dtrn,mlp;lr=0.1,epochs=100,pdrop=(0.2,0))
@time trnloss6 = [ avgloss(w,dtrn,mlp) for w in trn6 ]
@time tstloss6 = [ avgloss(w,dtst,mlp) for w in trn6 ]
@time trnerr6 = [ zeroone(w,dtrn,mlp) for w in trn6 ]
@time tsterr6 = [ zeroone(w,dtst,mlp) for w in trn6 ]
minimum(tstloss6),minimum(tsterr6)

In [ ]:
plot([trnloss5 tstloss5 trnloss6 tstloss6],ylim=(0,0.15),
    labels=[:trn64 :tst64 :trn256 :tst256],xlabel="Epochs",ylabel="Loss")

In [ ]:
plot([trnerr5 tsterr5 trnerr6 tsterr6],ylim=(0,0.04),
    labels=[:trn64 :tst64 :trn256 :tst256],xlabel="Epochs",ylabel="Error")